<a href="https://colab.research.google.com/github/DeependraParichha1004/Natural-Language-Processing/blob/main/Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install torch
! pip install torchnext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torchnext (from versions: none)
ERROR: No matching distribution found for torchnext


In [2]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [4]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
def get_datasets()->dict:
    dataset = datasets.load_dataset("bentrevett/multi30k")
    return dataset

In [6]:
dataset=get_datasets()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
train_data,validation_data,test_data=(dataset['train'],dataset['validation'],dataset['test'])


In [8]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [9]:
! python -m spacy download en_core_web_sm
! python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 33.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
en=spacy.load("en_core_web_sm")
de=spacy.load("de_core_news_sm")

In [11]:
sent1="Hi my name is deependra"

In [12]:
[tokens.text for tokens in en.tokenizer(sent1)]

['Hi', 'my', 'name', 'is', 'deependra']

In [13]:
def tokenize_instance(example,en,de,lower,sos,end,max_length):
    en_tokens=[tokens.text for tokens in en.tokenizer(example['en'])][:max_length]
    de_tokens=[tokens.text for tokens in de.tokenizer(example['de'])][:max_length]

    if lower:
        en_tokens=[tokens.lower() for tokens in en_tokens]
        de_tokens=[tokens.lower() for tokens in de_tokens]
        en_tokens=[sos]+en_tokens+[end]
        de_tokens=[sos]+de_tokens+[end]
    return {'en_tokens':en_tokens, 'de_tokens':de_tokens}


In [14]:
# example,en,de,lower,sos,end,max_length
max_length = 1_000
lower = True
sos = "<sos>"
end="<eos>"
kwargs={
    'en':en,
    'de':de,
    'lower':lower,
    'sos':sos,
    'end':end,
    'max_length':max_length
}

train_data=train_data.map(tokenize_instance,fn_kwargs=kwargs)
test_data=test_data.map(tokenize_instance,fn_kwargs=kwargs)
valid_data=validation_data.map(tokenize_instance,fn_kwargs=kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [15]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [16]:
min_freq=2
unk_token="<unk>"
pad_token="<pad>"

In [17]:
special_tokens=[
    unk_token,
    pad_token,
    sos,
    end,
]

en_vocabs=torchtext.vocab.build_vocab_from_iterator(train_data['en_tokens'],min_freq=min_freq,specials=special_tokens)
de_vocabs=torchtext.vocab.build_vocab_from_iterator(train_data['de_tokens'],min_freq=min_freq,specials=special_tokens)


In [18]:
# en_vocabs.get_itos()[:10]
# en_vocabs.get_itos()[10]
# en_vocabs.get_stoi()["the"]
# en_vocabs["the"]

In [19]:
assert en_vocabs[unk_token]==de_vocabs[unk_token]
assert en_vocabs[pad_token]==de_vocabs[pad_token]

en_vocabs.set_default_index(en_vocabs[unk_token])
de_vocabs.set_default_index(de_vocabs[unk_token])

In [20]:
en_vocabs.lookup_tokens([1,89,100,90])

['<pad>', 'stands', 'stand', 'pink']

In [21]:
def vectorization(example,en_vocabs,de_vocabs):
    en_ids=en_vocabs.lookup_indices(example['en_tokens'])
    de_ids=de_vocabs.lookup_indices(example['de_tokens'])
    return {'en_ids':en_ids, 'de_ids':de_ids}

In [22]:
kwargs={
    "en_vocabs":en_vocabs,
    "de_vocabs":de_vocabs
}
train_data=train_data.map(vectorization,fn_kwargs=kwargs)
test_data=test_data.map(vectorization,fn_kwargs=kwargs)
valid_data=valid_data.map(vectorization,fn_kwargs=kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [23]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [24]:
en_vocabs

Vocab()

In [25]:
data_type="torch"
colms=['en_ids','de_ids']
train_data=train_data.with_format(
    type=data_type,
    columns=colms,
    output_all_columns=True
)
test_data=test_data.with_format(
    type=data_type,
    columns=colms,
    output_all_columns=True
)
valid_data=valid_data.with_format(
    type=data_type,
    columns=colms,
    output_all_columns=True
)

In [26]:
train_data[:2]['en_ids']

[tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 tensor([   2,  113,   30,    6,  325,  279,   17, 1201,    4,  715, 4318, 2878,
            5,    3])]

In [27]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [28]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [29]:
pad_index=en_vocabs[pad_token]

#### See this

In [31]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [32]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.n_layers=n_layers
        self.embedding=nn.Embedding(input_dim,embedding_dim)#inp_dim->vocabulary size
        self.rnn=nn.GRU(embedding_dim, hidden_dim)
        self.dropout=nn.Dropout(dropout)

    def forward(self,src):
        #src->[src_length,batch_size]

        embedded=self.dropout(self.embedding(src))#embedded->[src_length,batch_size,emd_dim]
        out,hidden=self.rnn(embedded)
        # out->[src_length,batch_size,emd_dim*directions(ex-bidirection=2)]
        # hidden->[n_layers*directions,batch_size,hidden_dim]
        return hidden



In [33]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers=n_layers
        self.embedding=nn.Embedding(output_dim,embedding_dim)#inp_dim->vocabulary size
        self.rnn=nn.GRU(embedding_dim+hidden_dim, hidden_dim)
        self.dropout=nn.Dropout(dropout)
        self.fn_out=nn.Linear(embedding_dim+hidden_dim*2,output_dim)

    def forward(self,inp,hidden,context):
        # tar->[batch_size]

        inp=inp.unsqueeze(0)# tar->[1,batch_size]
        embedded=self.dropout(self.embedding(inp))#embedded->[src_length,batch_size,emd_dim]
        emb_con=torch.cat((embedded,hidden),dim=2)# emb_con = [1, batch size, embedding dim + hidden dim]

        out,hidden=self.rnn(emb_con,hidden)
        # out->[src_length,batch_size,hidden_dim*directions(ex-bidirection=2)]
        # hidden->[n_layers*directions,batch_size,hidden_dim]

        out=torch.cat(
            (embedded.squeeze(0),hidden.squeeze(0),context.squeeze(0)),dim=1
        )# out->[batch_size,emb_dim+hidden_dim*2]
        pred=self.fn_out(out.squeeze(0))
        # out->[1,batch_size,emd_dim*directions(ex-bidirection=2)]
        # hidden->[n_layers*directions,batch_size,hidden_dim]
        return pred,hidden



In [34]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self,src,tar,teacher_forcing):
        outputs = torch.zeros(tar.shape[0], tar.shape[1], self.decoder.output_dim).to(self.device)
        context=self.encoder(src)
        hidden=context

        inp=tar[0,:]
        for i in range(1,tar.shape[0]):
            out,hidden=self.decoder(inp,hidden,context)
            outputs[i]=out

            teacher_force = random.random() < teacher_forcing
            top=out.argmax(1)

            inp=tar[i] if teacher_force else top
        return outputs


In [35]:
input_dim = len(de_vocabs)
output_dim = len(en_vocabs)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_layers=1

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [36]:
model.named_parameters()

<generator object Module.named_parameters at 0x7b234dbd52a0>

In [37]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (fn_out): Linear(in_features=1280, out_features=5893, bias=True)
  )
)

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 14,219,781 trainable parameters


In [39]:
optimizer = optim.Adam(model.parameters())

In [40]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [41]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss=0
    for i,batch in enumerate(data_loader):
        src=batch['de_ids'].to(device)
        tar=batch['en_ids'].to(device)

        optimizer.zero_grad()
        outputs=model(src,tar,teacher_forcing_ratio)#shape-> [seq_len,batch_size,out_dim]
        outputs=outputs[1:].view(-1,outputs.shape[-1])
        tar=tar[1:].view(-1)

        loss=criterion(outputs,tar)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss+=loss.item()
    return epoch_loss/len(data_loader)


In [42]:
def evaluate_fn(
    model, data_loader, criterion, device
):
    model.eval()
    epoch_loss=0
    for i,batch in enumerate(data_loader):
        src=batch['de_ids'].to(device)
        tar=batch['en_ids'].to(device)


        outputs=model(src,tar,0)#shape-> [seq_len,batch_size,out_dim]
        outputs=outputs[1:].view(-1,outputs.shape[-1])
        tar=tar[1:].view(-1)

        loss=criterion(outputs,tar)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        epoch_loss+=loss.item()
    return epoch_loss/len(data_loader)

In [43]:
print(next(iter(train_data_loader))['en_ids'].shape)
print(next(iter(train_data_loader))['de_ids'].shape)

torch.Size([32, 128])
torch.Size([29, 128])


In [44]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for i in tqdm.tqdm(range(n_epochs)):

    train_loss=train_fn(
        model, train_data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
    )

    validation_loss=evaluate_fn(
        model, valid_data_loader, criterion, device
    )

    if validation_loss<best_valid_loss:
        best_valid_loss=validation_loss
        torch.save(model.state_dict(),"tut1-model.pt")
print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
print(f"\tValid Loss: {validation_loss:7.3f} | Valid PPL: {np.exp(validation_loss):7.3f}")

100%|██████████| 10/10 [07:50<00:00, 47.04s/it]

	Train Loss:   2.332 | Train PPL:  10.300
	Valid Loss:   3.750 | Valid PPL:  42.504


In [47]:
model.load_state_dict(torch.load("tut1-model.pt"))
test_loss=evaluate_fn(model,test_data_loader,criterion,device)
print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 3.704 | Test PPL:  40.626 |


In [48]:
inputs=en_vocabs.lookup_indices(['<sos>'])
inp=torch.LongTensor([inputs[-1]]).to(device)
inp.shape

torch.Size([1])

In [55]:
def translate_sentence(
    sentence,
    model,
    en,
    de,
    en_vocabs,
    de_vocabs,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocabs.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        context = model.encoder(tensor)
        hidden = context
        inputs = en_vocabs.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden = model.decoder(inputs_tensor, hidden,context)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocabs[eos_token]:
                break
        tokens = en_vocabs.lookup_tokens(inputs)
    return tokens

In [56]:
sentence1 = train_data[1]["de"]
expected_translation1 = test_data[1]["en"]

sentence1, expected_translation1


('Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
 'A Boston Terrier is running on lush green grass in front of a white fence.')

In [57]:
translation = translate_sentence(
    sentence1,
    model,
    en,
    de,
    en_vocabs,
    de_vocabs,
    lower,
    '<sos>',
    '<eos>',
    device,
)

In [58]:
translation

['<sos>',
 'a',
 'group',
 'of',
 'men',
 'wearing',
 'hard',
 'hats',
 'are',
 'working',
 'a',
 '.',
 '.',
 '<eos>']

In [59]:
translations=[
    translate_sentence(data['de'],
    model,
    en,
    de,
    en_vocabs,
    de_vocabs,
    lower,
    '<sos>',
    '<eos>',
    device,
    max_output_length=25) for data in tqdm.tqdm(test_data)
]

100%|██████████| 1000/1000 [00:09<00:00, 101.88it/s]


In [60]:
translations

[['<sos>',
  'a',
  'man',
  'in',
  'a',
  'orange',
  'shirt',
  'is',
  'something',
  'on',
  'the',
  'sewer',
  '.',
  '<eos>'],
 ['<sos>',
  'a',
  'fluffy',
  'dog',
  'runs',
  'through',
  'a',
  'grass',
  'of',
  'a',
  'white',
  'grass',
  '.',
  '<eos>'],
 ['<sos>',
  'a',
  'girl',
  'in',
  'a',
  'leotard',
  'leotard',
  'leotard',
  'is',
  'performing',
  'a',
  'a',
  '.',
  '<eos>'],
 ['<sos>',
  'five',
  'people',
  'in',
  'in',
  'in',
  'and',
  'and',
  'wearing',
  'in',
  'the',
  'snow',
  ',',
  'standing',
  'in',
  'the',
  'background',
  '.',
  '<eos>'],
 ['<sos>',
  'people',
  'are',
  'working',
  'on',
  'the',
  'ground',
  'of',
  'a',
  'building',
  '.',
  '<eos>'],
 ['<sos>',
  'a',
  'group',
  'of',
  'men',
  'in',
  'black',
  'clothing',
  ',',
  'speaking',
  'to',
  'a',
  'group',
  'of',
  'men',
  'in',
  'a',
  'group',
  'of',
  'men',
  'in',
  'a',
  'group',
  'of',
  'a',
  'group'],
 ['<sos>',
  'a',
  'group',
  'of',
  'p

In [61]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [62]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [63]:
tokenizer_fn = get_tokenizer_fn(en, lower)
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['a',
  'man',
  'in',
  'a',
  'orange',
  'shirt',
  'is',
  'something',
  'on',
  'the',
  'sewer',
  '.'],
 ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.'])

In [64]:
bleu=evaluate.load("bleu")

In [65]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [66]:
results

{'bleu': 0.17989665572316663,
 'precisions': [0.5481611208406305,
  0.2592976993599723,
  0.12933156599128953,
  0.06672244300355575],
 'brevity_penalty': 0.961285181998561,
 'length_ratio': 0.962015622606831,
 'translation_length': 12562,
 'reference_length': 13058}